# QuTiP example: Landau-Zener-Stuckelberg inteferometry

In [11]:
%matplotlib inline

In [12]:
import matplotlib.pyplot as plt

In [13]:
import numpy as np

In [14]:
from qutip import *
from qutip.ui.progressbar import TextProgressBar as ProgressBar

Landau-Zener-Stuckelberg interferometry: Steady state of a strongly driven two-level system, using the one-period propagator. 

In [15]:
# set up the parameters and start calculation
delta  = 1.0 * 2 * np.pi  # qubit sigma_x coefficient
w      = 2.0 * 2 * np.pi  # driving frequency
T      = 2 * np.pi / w     # driving period 

gamma1 = 0.0125        # relaxation rate
gamma2 = 1.125         # dephasing  rate

eps_list = np.linspace(-50.0, 50.0, 201) * 2 * np.pi
A_list   = np.linspace(  0.0, 50.0, 201) * 2 * np.pi
t_list   = np.linspace(  0.0, 20.0, 101) * 2 * np.pi

# pre-calculate the necessary operators
sx = sigmax(); sz = sigmaz(); sm = destroy(2); sn = num(2)

# collapse operators
c_op_list = [np.sqrt(gamma1) * sm, np.sqrt(gamma2) * sz]  # relaxation and dephasing

In [16]:
# ODE settings (for list-str format)
options = Options()
options.rhs_reuse = True
options.atol = 1e-6 # reduce accuracy to speed
options.rtol = 1e-5 # up the calculation a bit

In [17]:
# for function-callback style time-dependence
def hamiltonian_t(t, args):
    """ evaluate the hamiltonian at time t. """
    H0 = args[0]
    H1 = args[1]
    w  = args[2]
    return H0 + H1 * np.sin(w*t)

In [18]:
# perform the calculation for each combination of eps and A, store the result
# in a matrix
def calculate():

    p_mat = np.zeros((len(eps_list), len(A_list)))

    pbar = ProgressBar(len(eps_list))
    
    for m, eps in enumerate(eps_list):
        H0 = delta * sx + eps * sz

        pbar.update(m)

        for n, A in enumerate(A_list):
            H1 = A * sz

            # function callback format
            #args = (H0, H1, w); H_td = hamiltonian_t

            # list-str format
            #args = {'w': w}; H_td = [H0, [H1, 'sin(w * t)']]

            # list-function format
            args = w; H_td = [H0, [H1, lambda t, w: np.sin(w * t)]]

            U = propagator(H_td, T, c_op_list, args, options)
            rho_ss = propagator_steadystate(U)

            p_mat[m,n] = np.real(expect(sn, rho_ss))

    return p_mat

In [19]:
p_mat = calculate()

/usr/local/lib/python3.5/dist-packages/scipy/integrate/_ode.py:927: UserWarning: zvode: Excess work done on this call. (Perhaps wrong MF.)
  self.messages.get(istate, unexpected_istate_msg)))


Exception: ODE integration error: Try to increase the allowed number of substeps by increasing the nsteps parameter in the Options class.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

A_mat, eps_mat = np.meshgrid(A_list/(2*np.pi), eps_list/(2*np.pi))

mp = ax.pcolor(eps_mat, A_mat, p_mat)
ax.set_xlabel(r'Bias point $\epsilon$')
ax.set_ylabel(r'Amplitude $A$')
ax.set_title("Steadystate excitation probability\n" +
             r'$H = -\frac{1}{2}\Delta\sigma_x -\frac{1}{2}\epsilon\sigma_z - \frac{1}{2}A\sin(\omega t)$' + "\n")
plt.colorbar(mp);

## Versions

In [ ]:
from qutip.ipynbtools import version_table

version_table()